### Phương pháp Đơn hình Dantzig (Trường hợp bi > 0)

In [3]:
import numpy as np
from sympy import*
import math

In [4]:
from decimal import Decimal, getcontext 

# Set global precision for decimal calculation
getcontext().prec = 20

### Sử dụng thư viện Scipy

In [5]:
import numpy as np
from scipy.optimize import linprog

In [6]:
A = np.array([[0.5, -5.5, -2.5, 9], [0.5, -1.5, -0.5, 1], [1, 0, 0, 0]])
c = np.array([-10, 57, 9, 24])
b = np.array([0, 0 ,1])
bounds = [(0, None), (0, None), (0, None), (0, None)]
constraints = [{'type': 'ineq', 'fun': lambda x: b - A.dot(x)}]

In [7]:
res = linprog(c, A_ub=A, b_ub=b, method='highs')

In [8]:
print(res)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -1.0
              x: [ 1.000e+00  0.000e+00  1.000e+00  0.000e+00]
            nit: 1
          lower:  residual: [ 1.000e+00  0.000e+00  1.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  3.000e+01  0.000e+00  4.200e+01]
          upper:  residual: [       inf        inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 2.000e+00  0.000e+00  0.000e+00]
                 marginals: [-0.000e+00 -1.800e+01 -1.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0


### Sử dụng thư viện để kiểm tra

In [9]:
from scipy.optimize import linprog
import numpy as np
# Hàm mục tiêu và ràng buộc của bài toán tối ưu hóa
#c = [1, -2, 3]
#A = [[-1, 2, -3], [4, -5, 6], [-7, 8, -9]]
#b = [-1, 2, -3]
A = np.array([[-3,-1], [-1,-2], [1,-1], [1,0], [0,1]])
c = np.array([-1,1])
b = np.array([-3.,-4.,1.,5.,5.])


# Giải bài toán tối ưu hóa bằng phương pháp đơn hình Bland
# res = linprog(c, A_ub=A, b_ub=b, method='highs', options={'bland': True})
res = linprog(c, A_ub=A, b_ub=b, method='highs', options={'presolve': True})


# In kết quả
print('Nghiệm tối ưu: ', res.x)
print('Giá trị hàm mục tiêu tối ưu: ', res.fun)

Nghiệm tối ưu:  [5. 4.]
Giá trị hàm mục tiêu tối ưu:  -1.0


In [10]:
import numpy as np

# A: ma trận chưa hệ của của các biến trong các ràng buộc
# b: mảng chứa các giá trị bi
# c: mảng chứa hệ số các biến trong hàm mục tiêu

def dantzig_method(A, b, c):
    m, n = A.shape

    # Khởi tạo biến giá trị tối ưu
    z = 0 
    
    # Tính ma trận bổ sung
    T = np.hstack((A, np.eye(m)))
    c_n = np.concatenate((c, np.zeros(m)))
    b_n = b.copy()
    
    # Copy biến ban đầu để so sánh
    A_cop = T.copy()
    c_cop = c_n.copy()
    b_cop = b.copy()
    
    # Tính biến cơ sở ban đầu
    basis = list(range(n, n + m))
    # Chuyển về dạng chuẩn tắc
    while True:
        # Tìm biến vào
        j = np.argmin(c_n)
        
        if c_n[j] >= 0:
            break
            
        # Tìm biến ra
        ratios = [float('inf')] * m
        for i in range(m):
            if T[i, j] > 0:
                ratios[i] = b_n[i] / T[i, j]
        
        i = np.argmin(ratios)

        # Kiểm tra bài toán không giới nội
        if ratios[i] == float('inf'):
            print("Bài toán không giới nội.")
            return -np.inf, None
        
        # Tính giá trị tối ưu
        z += c_n[j] / T[i, j] * b_n[i]
        # Cập nhật ma trận
        basis[i] = j
        
        for k in range(m):
            if k != i:
                b_n[k] -= T[k, j]/T[i,j] * b_n[i]
                T[k, :] -=  T[k, j] / T[i,j] * T[i, :]
        
        b_n[i] /= T[i, j]
        T[i, :] /= T[i, j]       
        c_n -= c_n[j] * T[i, :]
        # Kiểm tra thuật toán có hoàn thành hay không 
        # Nếu Từ vựng mới trùng với từ vựng ban đầu thì dừng thuật toán
        if (c_n == c_cop).all() and (T == A_cop).all() and (b_n == b_cop).all():
            print("Thuật toán lặp vô hạn.")
            return None, None
        
    # Tìm nghiệm tối ưu
    x = np.zeros(n)
    for i in range(m):
        if basis[i] < n:
            x[basis[i]] = b_n[i]
                           
    return z, x 

#### bi >0

In [11]:
c = np.array([-3,-1, 0])
A = np.array([[1,2,0], [1,1,-1], [7,3,-5]])
b = np.array([5.,2.,20.])
z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

z =  -15.0
x =  [5. 0. 3.]


In [12]:
c = np.array([-4,-5])
A = np.array([[2,2], [1,0], [0,1]])
b = np.array([9.,4.,3.])
z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

z =  -21.0
x =  [1.5 3. ]


In [13]:
A = np.array([[-3,-1], [-1,-2], [1,-1], [1,0], [0,1]])
c = np.array([-1,1])
b = np.array([-3.,-4.,1.,5.,5.])

z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

z =  -1.0
x =  [1. 0.]


#### Trường hợp không giới nội

In [14]:
c = np.array([-1,-3,1])
A = np.array([[2,2,-1],[3,-2,1], [1,-3,1]])
b = np.array([10.,10., 10.])
#c = np.array([-4, -5, 5])
#A = np.array([[1,2,-2], [1,-1, 1], [2,1,-1], [3,4,-4]])
#b = np.array([3,2,3,8])
z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

Bài toán không giới nội.
z =  -inf
x =  None


#### Trường hợp vô số nghiệm

#### Trường hợp lặp vô hạn

In [16]:
A = np.array([[0.5, -5.5, -2.5, 9], [0.5, -1.5, -0.5, 1], [1, 0,0,0]])
c = np.array([-10, 57, 9, 24])
b = np.array([0.,0.,1.])
z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

Thuật toán lặp vô hạn.
z =  None
x =  None


In [17]:
import numpy as np

A = np.array([[-1,0,1], [0,3,-3], [1,1,-1], [4,-1,1], [5, -1, 0]])
Q, R = np.linalg.qr(A)

if Q.shape[1] == A.shape[1]:
    print("Ma trận A khả giải.")
else:
    print("Ma trận A không khả giải.")

Ma trận A khả giải.



### Trường hợp 2 pha

In [136]:
def two_phases(A,b,c):

    m, n = A.shape
    # Khởi tạo biến giá trị tối ưu
    z = 0
    # Tính ma trận bổ sung có chứa x0
    c1 = np.concatenate((np.ones(1), np.zeros(m+n)))
    new_col = (-1*np.ones(m)).reshape(-1, 1)
    T = np.hstack((new_col,np.hstack((A, np.eye(m)))))
    b_n = b.copy()


    # Copy biến ban đầu để so sánh
    A_cop = T.copy()
    c_cop = c1.copy()
    b_cop = b.copy()

    # Tính biến cơ sở ban đầu
    basis = list(range(n+1, n + m+1))
    # Chuyển về dạng chuẩn tắc

    #Chọn biến vào
    j = 0
    #Tìm biến ra
    i = np.argmin(b_n)

    
    # Tính giá trị tối ưu
    z += c1[j] / T[i, j] * b_n[i]
    # Cập nhật ma trận
    basis[i] = j
    
    for k in range(m):
        if k != i:
            b_n[k] -= T[k, j]/T[i,j] * b_n[i]
            T[k, :] -=  T[k, j] / T[i,j] * T[i, :]
    
    b_n[i] /= T[i, j]
    T[i, :] /= T[i, j]       
    c1 -= c1[j] * T[i, :]

    #phase 1
    while True:
        # Tìm biến vào
        j = np.argmin(c1)
        
        if c1[j] >= 0:
            break
            
        # Tìm biến ra
        ratios = [float('inf')] * m
        for i in range(m):
            if T[i, j] > 0:
                ratios[i] = b_n[i] / T[i, j]
        
        i = np.argmin(ratios)

        # Kiểm tra bài toán không giới nội hoặc từ vựng tối ưu
        if ratios[i] == float('inf'):
            return -np.inf, None
        
        # Tính giá trị tối ưu
        z += c1[j] / T[i, j] * b_n[i]
        # Cập nhật ma trận
        basis[i] = j
        
        for k in range(m):
            if k != i:
                b_n[k] -= T[k, j]/T[i,j] * b_n[i]
                T[k, :] -=  T[k, j] / T[i,j] * T[i, :]
        
        b_n[i] /= T[i, j]
        T[i, :] /= T[i, j]       
        c1 -= c1[j] * T[i, :]
        # Kiểm tra thuật toán có hoàn thành hay không 
        # Nếu Từ vựng mới trùng với từ vựng ban đầu thì dừng thuật toán
        if (c1 == c_cop).all() and (T == A_cop).all() and (b_n == b_cop).all():
            print("Thuật toán lặp vô hạn.")
            return None, None
        
    #Kiểm tra bài toán có nghiệm khi hoàn thành phase 1
    if np.array_equal(c1,c_cop) == False:
        return None, None
    #Thiết lập hàm mục tiêu cho phase 2
    # c_n = np.concatenate((np.zeros(len(c)), np.zeros(m)))
    c_n = np.zeros(m+n) # trường hợp các biến trên hàm mục tiêu không đủ

    #loại bỏ x0
    T = T[:, 1:]
    T_z = T.copy()
    for i in range(m):
        if basis[i] <= n:
            T_z[i,basis[i]-1] = 0 # làm mất biến thay vào khi thêm vào hàm mục tiêu
            c_n += c[basis[i]-1] * T_z[i,:] #vì basis có tính 0
            z += c[basis[i]-1] * b_n[i]
    #Cập nhật giá trị z cho phase 2
    c_n *= -1
    #Cập nhật lại basis
    basis -= np.ones(len(basis))
    #phase 2

    while True:
        # Tìm biến vào
        j = np.argmin(c_n)
        
        if c_n[j] >= 0:
            break
            
        # Tìm biến ra
        ratios = [float('inf')] * m
        for i in range(m):
            if T[i, j] > 0:
                ratios[i] = b_n[i] / T[i, j]
        
        i = np.argmin(ratios)

        # Kiểm tra bài toán không giới nội
        if ratios[i] == float('inf'):
            print("Bài toán không giới nội.")
            return -np.inf, None
        
        # Tính giá trị tối ưu
        z += c_n[j] / T[i, j] * b_n[i]
        # Cập nhật ma trận
        basis[i] = j
        
        for k in range(m):
            if k != i:
                # x1 = T[k, j]/T[i,j]
                # x2 = b_n[i]
                # b_n[k] -= np.multiply(x1,x2)
                b_n[k] -= (T[k, j]/T[i,j]) * b_n[i]
                T[k, :] -=  T[k, j] / T[i,j] * T[i, :]
        
        b_n[i] /= T[i, j]
        T[i, :] /= T[i, j]
        c_n -= c_n[j] * T[i, :]
        # Kiểm tra thuật toán có hoàn thành hay không 
        # Nếu Từ vựng mới trùng với từ vựng ban đầu thì dừng thuật toán
        if np.array_equal(c_n, c_cop) and np.array_equal(T, A_cop) and np.array_equal(b_n, b_cop):
            print("Thuật toán lặp vô hạn.")
            # Thêm bland vào đây
            return None, None
        
    # Tìm nghiệm tối ưu
    x = np.zeros(n)
    for i in range(m):
        if basis[i] < n:
            if i <= len(b_n):
                x[int(basis[i])] = b_n[i]
            else:
                x[int(basis[i])] = 0
            # try:
            #     idx = int(basis[i])
            #     if idx < 0 or idx >= len(x):
            #         raise IndexError
            #     x[idx] = b_n[i]
            # except (IndexError, ValueError):
            #     print(f"Invalid basis index at i = {i}")
            #     x[basis[i]] = 0      




    return z, x  


In [137]:
A = np.array([[-1,-1,-1], [2,-1,1]])
c = np.array([-2,6,0])
b = np.array([-2,1])
b = b.astype(np.float64)
z, x = two_phases(A, b, c)
z = round(z,3)
print("z = ", z)
print("x = ", x)

z =  3.0
x =  [0.  0.5 1.5]


In [138]:
A = np.array([[-3,-1], [-1,-2], [1,-1], [1,0], [0,1]])
c = np.array([-1,1])
b = np.array([-3.,-4.,1.,5.,5.])
b = b.astype(np.float64)
z, x = two_phases(A, b, c)
z = round(z,3)
print("z = ", z)
print("x = ", x)

z =  -1.0
x =  [2. 1.]


In [139]:
A = np.array([[-4,1], [1,1], [-1,-1], [-3,2]])
c = np.array([23,-7])
b = np.array([-2,5.,-1.,1.])
b = b.astype(np.float64)
z, x = two_phases(A, b, c)
z = round(z,3)
print("z = ", z)
print("x = ", x)

z =  9.0
x =  [1. 2.]


### Trường hợp vô nghiệm

In [142]:
A = np.array([[-1,-1], [1,1]])
c = np.array([23,-7])
b = np.array([- 5.,1.])
b = b.astype(np.float64)
z, x = two_phases(A, b, c)
if (z != None):
    z = round(z,3)
print("z = ", z)
print("x = ", x)

z =  None
x =  None


### Trường hợp bài toán không giới nội

In [143]:
A = np.array([[-2,1], [-1,-2]])
c = np.array([1,-1])
b = np.array([-1.,-2.])
b = b.astype(np.float64)
z, x = two_phases(A, b, c)
if (z != None):
    z = round(z,3)
print("z = ", z)
print("x = ", x)

Bài toán không giới nội.
z =  -inf
x =  None
